In [2]:
import os
import boto3
import sys
import json
from strands import Agent, tool
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
sys.path.append("../interactive_tools")
from browser_viewer import BrowserViewerServer
import tempfile
import uuid
from datetime import datetime
from rich.console import Console
import time
import requests

# Set AWS region
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')
REGION = os.environ['AWS_DEFAULT_REGION']

# # Import utils (adjust path as needed)
# Add current directory to Python path explicitly
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

print(f"Current directory: {current_dir}")
print(f"Python path includes current dir: {current_dir in sys.path}")

# Now try importing
import techresidential_utils as utils
print("✅ utils imported successfully")

Current directory: /Users/hsin-weilin/Desktop/projects/AWS_AgentCore_TechResidential/agents
Python path includes current dir: True
✅ utils imported successfully


In [76]:
# --- MCP Client Setup for AgentCore Gateway ---

REGION = "us-east-1"
USER_POOL_ID = "us-east-1_VKErYTcMn"
USER_POOL_NAME = "techresidential-agentcore--user-pool"
RESOURCE_SERVER_ID = "techresidential-gateway-id"  
RESOURCE_SERVER_NAME = "techresidential-gateway-name"
CLIENT_NAME = "techresidential-agentcore-gateway-client"

SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]

scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

# Get Gateway URL
# Create an SSM client
ssm = boto3.client('ssm', region_name=REGION)  # Change region if needed
# Retrieve the parameter value
response = ssm.get_parameter(
    Name="AgentCore_Gateway_URL",
    WithDecryption=False  # Set to True if the parameter is a SecureString
)
gateway_url = response['Parameter']['Value']
response2 = ssm.get_parameter(
    Name="NOVA_ACT_API_KEY",
    WithDecryption=True  # Set to True if the parameter is a SecureString
)
nova_act_api_key = response2['Parameter']['Value']

# Get Cognito User Pool Client Secret from Secret Manager
print("Retrieving Cognito User Pool client secret from Secret Manager")
secrets_client = boto3.client('secretsmanager', region_name=REGION)
secret_response = secrets_client.get_secret_value(SecretId='prod/Cognito-Agentcore-Gateway-Client')
secret_dict = json.loads(secret_response['SecretString'])
client_id = secret_dict['client_id']
client_secret = secret_dict['client_secret']

# # Get Client access token from Cognito
print("Requesting access token from Cognito...")
token_response = utils.get_token(USER_POOL_ID, client_id, client_secret, scopeString, REGION)
token = token_response["access_token"]
print("Access token obtained successfully", token)

Retrieving Cognito User Pool client secret from Secret Manager
Requesting access token from Cognito...
Access token obtained successfully eyJraWQiOiJhTUtWVDhINWtHU2JHWlYrMmtTOUt1bEVBVjl3OXpZS0g3VzRuYStPYzB3PSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIzY3I0cm03cmpqOGhnNXN1bWhxa2htMmcxcCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoidGVjaHJlc2lkZW50aWFsLWdhdGV3YXktaWRcL2dhdGV3YXk6d3JpdGUgdGVjaHJlc2lkZW50aWFsLWdhdGV3YXktaWRcL2dhdGV3YXk6cmVhZCIsImF1dGhfdGltZSI6MTc1OTUzNDc0MiwiaXNzIjoiaHR0cHM6XC9cL2NvZ25pdG8taWRwLnVzLWVhc3QtMS5hbWF6b25hd3MuY29tXC91cy1lYXN0LTFfVktFcllUY01uIiwiZXhwIjoxNzU5NTM4MzQyLCJpYXQiOjE3NTk1MzQ3NDIsInZlcnNpb24iOjIsImp0aSI6IjU0ZGU2M2EyLTMyODQtNGVhNi04ZjJhLWVkMDliN2ZiY2I0MSIsImNsaWVudF9pZCI6IjNjcjRybTdyamo4aGc1c3VtaHFraG0yZzFwIn0.Qhfw45_l8HvanFt3Y10wSs1RE4a8AEUQaaLLXAUqjR-KoW1251dEjXRom57nUQBjTJPuJ4PQG9MsAkK2C8KI6-2_p1_dhmFJrerXjESpAOb-R_z--UvZqM-tbVMfIms_CF7F8FjHOe5Kp_hidonoqoefEFD4OWPLsGq2N9IjBIgLk5GffdtqnZLKBUV936kNHASheNWuVyyg3j-19GrhLM7V86Pb9j-yOKtxg5pwwTw2G-f4saPWALFOhBElJFh-tHvlvq

In [16]:
@tool
def browse_web(instruction: str, starting_url: str, system_prompt: str, client_name: str):
    """Use Nova Act to browse the web, download files, and upload them to S3
    
    Args:
        instruction: What you want to accomplish on the web
        starting_url: URL to start from
        system_prompt: System prompt for the agent
        client_name: Used to organize S3 uploads
    
    Returns:
        Result of the web browsing task including S3 link if files were downloaded
    """
    console = Console()
    
    # Create a temporary directory for saving downloads
    download_dir = os.path.join(tempfile.gettempdir(), "nova_act_downloads")
    os.makedirs(download_dir, exist_ok=True)
    console.print(f"[yellow]Created temporary directory: {download_dir}[/yellow]")

    # start browser session and download file & upload to s3
    try:
        with browser_session("us-east-1") as browser_client:
            ws_url, headers = browser_client.generate_ws_headers()
            
            try: 
                with NovaAct(
                    cdp_endpoint_url=ws_url,
                    cdp_headers=headers,
                    nova_act_api_key=nova_act_api_key,
                    starting_page=starting_url
                ) as nova_act:
                    prompt = system_prompt + "\nuser prompt: " + instruction
                    
                    result = nova_act.act(prompt)
                    console.print(result)
                    
                    # Extract the href of the <a download> link
                    download_url = nova_act.page.get_attribute("a[download]", "href")
                    
                    # Save to local temp dir
                    file_path = os.path.join(download_dir, os.path.basename(download_url))
                    
                    # Fetch directly
                    response = requests.get(download_url)
                    with open(file_path, "wb") as f:
                        f.write(response.content)
                    
                    console.print(f"✅ File saved at: {file_path}")

                    if os.path.exists(file_path):
                        file_size = os.path.getsize(file_path)
                        file_ext = os.path.splitext(file_path)[1]
                        print(f"✅ File downloaded: {file_path}")
                        print(f"Size: {file_size} bytes")
                        print(f"Extension: {file_ext}")
                        with open(file_path, "rb") as f:
                            preview = f.read(100)
                            print(f"First 100 bytes: {preview}")
                        # Upload to S3
                        s3 = boto3.client('s3', region_name=REGION)
                        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                        bucket_name = "bedrock-web-automation-dev-storage"
                        s3_file_key = f"downloaded-files/{client_name}/{timestamp}-{os.path.basename(file_path)}"
                        try:
                            s3.upload_file(
                                file_path,
                                bucket_name,
                                s3_file_key,
                                ExtraArgs={'ContentDisposition': f'attachment; filename="{os.path.basename(file_path)}"'}
                            )
                            # Generate pre-signed URL
                            presigned_url = s3.generate_presigned_url(
                                'get_object',
                                Params={'Bucket': bucket_name, 'Key': s3_file_key},
                                ExpiresIn=3600
                            )
                            print(f"✅ Uploaded to S3: {s3_file_key}")
                            print(f"Presigned URL (valid 1 hour): {presigned_url}")
                            #return successful result
                            return {
                                "status": "success",
                                "s3_key": s3_file_key,
                                "s3_url": presigned_url,
                                "file_name": os.path.basename(file_path),
                                "file_size": file_size,
                                "file_ext": file_ext
                            }
                        except Exception as s3_error:
                            print(f"❌ Error uploading to S3: {repr(s3_error)}")
                    else:
                        print(f"❌ File not found: {file_path}")
            except Exception as nova_error:
                error_msg = nova_error.args[0] if nova_error.args else ""
                console.print(f"[red]NovaAct error: {error_msg}[/red]")
                if "credentials are not valid" in error_msg:
                    return "Authentication failed: The provided credentials were rejected by the website."
                elif "captcha" in error_msg.lower():
                    return "Security challenge detected: The website requires human verification."
                elif "timeout" in error_msg.lower():
                    return "Operation timed out: The web task took too long to complete."
                else:
                    return f"Web automation error: {error_msg}"
    except Exception as e:
        console.print(f"[red]Error: {repr(e)}[/red]")
        import traceback
        console.print(traceback.format_exc())
        return f"Error browsing web: {repr(e)}"
    finally:
        # Clean up our temporary directory
        import shutil
        try:
            if os.path.exists(download_dir):
                shutil.rmtree(download_dir, ignore_errors=True)
        except Exception:
            pass

print("browse_web tool with download capability")

browse_web tool with download capability


In [3]:
@tool
def live_view_with_nova_act(instruction: str, starting_url: str, system_prompt: str, client_name: str):
    """Use Nova Act to browse the web and complete web-based tasks with live view"""
    # Add necessary imports
    import socket
    import os
    import tempfile
    import boto3
    import uuid
    from datetime import datetime
    from rich.console import Console
    import time
    console = Console()
    
    # Create a temporary directory for saving downloads
    download_dir = os.path.join(tempfile.gettempdir(), "nova_act_downloads")
    os.makedirs(download_dir, exist_ok=True)
    console.print(f"[yellow]Created temporary directory: {download_dir}[/yellow]")
    
    # S3 bucket configuration
    s3_bucket_name = "bedrock-web-automation-dev-storage/downloaded-files/" + client_name
    s3_region = os.environ.get("AWS_REGION", "us-east-1")
    
    # Simpler port check and selection
    def find_free_port(start=8000, max_attempts=20):
        """Find an available port starting from start_port"""
        for port in range(start, start + max_attempts):
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                if s.connect_ex(('localhost', port)) != 0:
                    return port
        return None
        
    # Find a free port
    port = find_free_port()
    if not port:
        console.print("[red]Could not find an available port[/red]")
        return "Error: Could not find an available port between 8000-8019"
    
    console.print(f"[green]Using port: {port}[/green]")
    
    try:
        # Step 1: Create browser session
        with browser_session(REGION) as client:
            ws_url, headers = client.generate_ws_headers()

            # Step 2: Start viewer server with our selected port
            viewer = BrowserViewerServer(client, port=port)
            viewer_url = viewer.start(open_browser=True)
            console.print(f"[green]Browser opened on port {port}[/green]")
            
            try: 
                with NovaAct(
                    cdp_endpoint_url=ws_url,
                    cdp_headers=headers,
                    nova_act_api_key=nova_act_api_key,
                    starting_page=starting_url
                ) as nova_act:
                    prompt = system_prompt + "\nuser prompt: " + instruction
                    
                    result = nova_act.act(prompt)
                    console.print(result)
                    
                    # Extract the href of the <a download> link
                    download_url = nova_act.page.get_attribute("a[download]", "href")
                    
                    # Save to local temp dir
                    file_path = os.path.join(download_dir, os.path.basename(download_url))
                    
                    # Fetch directly
                    response = requests.get(download_url)
                    with open(file_path, "wb") as f:
                        f.write(response.content)
                    
                    console.print(f"✅ File saved at: {file_path}")

                    if os.path.exists(file_path):
                        file_size = os.path.getsize(file_path)
                        file_ext = os.path.splitext(file_path)[1]
                        print(f"✅ File downloaded: {file_path}")
                        print(f"Size: {file_size} bytes")
                        print(f"Extension: {file_ext}")
                        with open(file_path, "rb") as f:
                            preview = f.read(100)
                            print(f"First 100 bytes: {preview}")
                        # Upload to S3
                        s3 = boto3.client('s3', region_name=REGION)
                        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                        bucket_name = "bedrock-web-automation-dev-storage"
                        s3_file_key = f"downloaded-files/{client_name}/{timestamp}-{os.path.basename(file_path)}"
                        try:
                            s3.upload_file(
                                file_path,
                                bucket_name,
                                s3_file_key,
                                ExtraArgs={'ContentDisposition': f'attachment; filename="{os.path.basename(file_path)}"'}
                            )
                            # Generate pre-signed URL
                            presigned_url = s3.generate_presigned_url(
                                'get_object',
                                Params={'Bucket': bucket_name, 'Key': s3_file_key},
                                ExpiresIn=3600
                            )
                            print(f"✅ Uploaded to S3: {s3_file_key}")
                            print(f"Presigned URL (valid 1 hour): {presigned_url}")
                            #return successful result
                            return {
                                "status": "success",
                                "s3_key": s3_file_key,
                                "s3_url": presigned_url,
                                "file_name": os.path.basename(file_path),
                                "file_size": file_size,
                                "file_ext": file_ext
                            }
                        except Exception as s3_error:
                            print(f"❌ Error uploading to S3: {repr(s3_error)}")
                    else:
                        print(f"❌ File not found: {file_path}")
            except Exception as nova_error:
                error_msg = nova_error.args[0] if nova_error.args else ""
                console.print(f"[red]NovaAct error: {error_msg}[/red]")
                if "credentials are not valid" in error_msg:
                    return "Authentication failed: The provided credentials were rejected by the website."
                elif "captcha" in error_msg.lower():
                    return "Security challenge detected: The website requires human verification."
                elif "timeout" in error_msg.lower():
                    return "Operation timed out: The web task took too long to complete."
                else:
                    return f"Web automation error: {error_msg}"
        
    except Exception as e:
        console.print(f"\n[red]Error: {e}[/red]")
        import traceback
        traceback.print_exc()
        return f"Browser session error: {str(e)}"
    finally:
        # Ensure we clean up properly
        console.print("\n\n[yellow]Shutting down...[/yellow]")
        if "client" in locals():
            client.stop()
            console.print("✅ Browser session terminated")

print("live browser tool")

live browser tool


In [84]:
@tool
def nova_act_download(instruction: str, starting_url: str, system_prompt: str, client_name: str):

    console = Console()

    with NovaAct(
        headless=False,
        nova_act_api_key=nova_act_api_key,
        starting_page=starting_url
    ) as nova_act:
        try:
            with nova_act.page.expect_download() as download_info:
                result = nova_act.act(instruction)
    
            if not download_info.value:
                return {"status": "no download event"}
            console.print(result)
    
            # Get a writable temp directory
            download_dir = tempfile.gettempdir()
            save_path = os.path.join(download_dir, "my_downloaded_file")
            # Now save the downloaded file permanently to a location of your choice.
            download_info.value.save_as(save_path)
            download_path = download_info.value.path()
            console.print(f"Downloaded file: {download_path}")
            
            # Print file size
            if os.path.exists(download_path):
                file_size = os.path.getsize(download_path)
                console.print(f"File size: {file_size} bytes")
            
                # Print first 100 bytes (as preview)
                with open(download_path, "rb") as f:
                    preview = f.read(100)
                    print(f"First 100 bytes: {preview}")
            
                # Print file extension
                file_ext = os.path.splitext(download_path)[1]
                console.print(f"File extension: {file_ext}")
                # Upload to S3
                s3 = boto3.client('s3', region_name=REGION)
                timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
                bucket_name = "bedrock-web-automation-dev-storage"
                s3_file_key = f"downloaded-files/{client_name}/{timestamp}-{os.path.basename(download_path)}"
                try:
                    s3.upload_file(
                        download_path,
                        bucket_name,
                        s3_file_key,
                        ExtraArgs={'ContentDisposition': f'attachment; filename="{os.path.basename(download_path)}"'}
                    )
                    # Generate pre-signed URL
                    presigned_url = s3.generate_presigned_url(
                        'get_object',
                        Params={'Bucket': bucket_name, 'Key': s3_file_key},
                        ExpiresIn=3600
                    )
                    console.print(f"✅ Uploaded to S3: {s3_file_key}")
                    console.print(f"Presigned URL (valid 1 hour): {presigned_url}")
                    #return successful result
                    return {
                        "status": "success",
                        "s3_key": s3_file_key,
                        "s3_url": presigned_url,
                        "file_name": os.path.basename(download_path)
                    }
                except Exception as s3_error:
                    console.print(f"❌ Error uploading to S3: {repr(s3_error)}")
                    return {
                        "status": "s3 error"
                    }
            else:
                console.print("Downloaded file does not exist.")
        except TimeoutError:
            return {"status": "timeout", "reason": "No download event within 30 seconds"}
        except Exception as e:
            return {"status": "error", "reason": str(e)}
        # if not download_info.value:
        #     console.print("[red]No download event was captured![/red]")
        #     return {"status": "error", "reason": "No download event"}
        # download_dir = tempfile.gettempdir()
        # # Get the original file name from the download object
        # original_filename = download_info.value.suggested_filename() or "downloaded_file"
        # save_path = os.path.join(download_dir, original_filename)
        
        # download_info.value.save_as(save_path)
        # download_path = save_path  # Use save_path for all further operations

        # console.print(f"Downloaded file: {download_path}")

        
        # # Print file size
        # if os.path.exists(download_path):
        #     file_size = os.path.getsize(download_path)
        #     file_ext = os.path.splitext(download_path)[1]
        #     console.print(f"✅ File downloaded: {download_path}")
        #     print(f"Size: {file_size} bytes")
        #     print(f"Extension: {file_ext}")
        #     with open(download_path, "rb") as f:
        #         preview = f.read(100)
        #         print(f"First 100 bytes: {preview}")
        #     # Upload to S3
        #     s3 = boto3.client('s3', region_name=REGION)
        #     timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        #     bucket_name = "bedrock-web-automation-dev-storage"
        #     s3_file_key = f"downloaded-files/{client_name}/{timestamp}-{os.path.basename(download_path)}"
        #     try:
        #         s3.upload_file(
        #             download_path,
        #             bucket_name,
        #             s3_file_key,
        #             ExtraArgs={'ContentDisposition': f'attachment; filename="{os.path.basename(download_path)}"'}
        #         )
        #         # Generate pre-signed URL
        #         presigned_url = s3.generate_presigned_url(
        #             'get_object',
        #             Params={'Bucket': bucket_name, 'Key': s3_file_key},
        #             ExpiresIn=3600
        #         )
        #         console.print(f"✅ Uploaded to S3: {s3_file_key}")
        #         console.print(f"Presigned URL (valid 1 hour): {presigned_url}")
        #         #return successful result
        #         return {
        #             "status": "success",
        #             "s3_key": s3_file_key,
        #             "s3_url": presigned_url,
        #             "file_name": os.path.basename(download_path)
        #         }
        #     except Exception as s3_error:
        #         console.print(f"❌ Error uploading to S3: {repr(s3_error)}")
        #         return {
        #             "status": "s3 error"
        #         }
        # else:
        #     print(f"❌ File not found: {download_path}")

print("nova_act_download_tool")

nova_act_download_tool


In [85]:
def create_streamable_http_transport():
    return streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {token}"})

# Create MCP client and discover tools from the gateway using "with" context manager
client = MCPClient(create_streamable_http_transport)

with client:
    gateway_tools = client.list_tools_sync()
    tools = gateway_tools + [nova_act_download]
    # now initialize your agent with these tools
    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    model = BedrockModel(model_id=model_id)
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt="""You're a helpful Web UI automation assistant. 
IMPORTANT BEHAVIOR RULES:
- Do NOT retry failed tool calls on your own. If a tool fails, explain the error and stop.
- If the web automation tool fails, do not attempt to call it again. Report the error only.
- After any error in web automation, provide a summary of what was completed and what failed.
- Never output the password in plain text in your responses.
- Never make multiple attempts to run the same web automation in a single response.

Follow this exact process:
   - Use the nova_act_download with FOUR arguments:
     a. instruction: click the Open in new tab button and RETURN
     b. starting_url: https://techresidential.com/hsinwei-download-demo.php
     c. system_prompt: Use this: "You are a helpful Web UI automation assistant. Follow the user prompt exactly. 
     Very important: Never click the download button twice. If you clicked the download button, RETURN ACTION COMPLETE in ActResult. Even if the page doens't change.
     Very important: If you find yourself repeating the same action (such as clicking a button, typing into a field, or submitting a form) more than 2 times without success, STOP the automation immediately and RETURN ACTION COMPLETE in ActResult
     d. client_name: the client_name that you are given (as a string)
"""
    )

    print(f"Available tools: {[tool.tool_name for tool in tools]}")

    # Test Invoke Agent
    response = agent("client_name=BlueHonuFire")

Available tools: ['CredentialsRetriever___get_credentials', 'PromptFileRetriever___get_prompt_file', 'nova_act_download']
I'll help you with this web automation task for BlueHonuFire client. I'll use the nova_act_download function to navigate to the specified URL and click the "Open in new tab" button.

Let me execute this for you:
Tool #1: nova_act_download



start session cf87d426-60fc-4c7a-8df0-ab95704ac72e on https://techresidential.com/hsinwei-download-demo.php logs dir /var/folders/h_/fl30x8w52y55k21qz1_pl8fm0000gn/T/tmpz6k2doxj_nova_act_logs/cf87d426-60fc-4c7a-8df0-ab95704ac72e

cf87> act("click the Open in new tab button and RETURN")
cf87> ...
cf87> think("I am on the Same-Domain Download Methods Demo page. My task is to click the 'Open in new tab' button and return. I can see the 'Open in new tab' button under the '1) Standard <a> link' section. I should click the 'Open in new tab' button to select it.");
cf87> agentClick(" <box>326,265,362,413</box> ");
cf87> ...
cf87> think("I am now on a new page with the techresidential logo, so my last action was successful. I have clicked the 'Open in new tab' button. My task is complete, and I was not asked to provide specific information, so I will simply return.");
cf87> return;

cf87> ** View your act run here: /var/folders/h_/fl30x8w52y55k21qz1_pl8fm0000gn/T/tmpz6k2doxj_nova_act_logs/cf8

It appears that the web automation encountered a timeout error while waiting for a download event. The automation attempted to click the "Open in new tab" button, but it seems that the expected action didn't complete successfully within the allocated time.

Error details:
- The system timed out after 30000ms (30 seconds) while waiting for a download event
- The automation was unable to complete the requested action

This could be due to several factors:
- The button might not have been found on the page
- The button might have been found but was not clickable
- The page might have loaded differently than expected
- There could be network connectivity issues

Since the web automation tool failed, I will not attempt to run it again. If you would like to try again or modify the approach, please let me know and I can help you with alternative solutions.

In [74]:
def create_streamable_http_transport():
    return streamablehttp_client(gateway_url, headers={"Authorization": f"Bearer {token}"})

# Create MCP client and discover tools from the gateway using "with" context manager
client = MCPClient(create_streamable_http_transport)

with client:
    gateway_tools = client.list_tools_sync()
    tools = gateway_tools + [nova_act_download]
    # now initialize your agent with these tools
    model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    model = BedrockModel(model_id=model_id)
    agent = Agent(
        model=model,
        tools=tools,
        system_prompt="""You're a helpful Web UI automation assistant. 
IMPORTANT BEHAVIOR RULES:
- Do NOT retry failed tool calls on your own. If a tool fails, explain the error and stop.
- If the web automation tool fails, do not attempt to call it again. Report the error only.
- After any error in web automation, provide a summary of what was completed and what failed.
- Never output the password in plain text in your responses.
- Never make multiple attempts to run the same web automation in a single response.

Follow this exact process:
        
1. FIRST STAGE - Data Collection:
   - When given a client_name, use the CredentialsRetriever___get_credentials tool to get login credentials
   - Then use the PromptFileRetriever___get_prompt_file tool to get the task instructions
   - Extract and clearly identify: website URL, username (could be email or something else), password, and task instructions

2. SECOND STAGE - Web Automation:
   - Use the nova_act_download with FOUR arguments:
     a. instruction: Include BOTH login details AND the task instructions
     b. starting_url: The URL extracted from credentials
     c. system_prompt: Use this: "You are a helpful Web UI automation assistant. Follow the user prompt exactly. 
     Very important: Whenever you clicked on the download button, wait for the file to downlaod and the web page to change.
     Very important: If you find yourself repeating the same action (such as clicking a button, typing into a field, or submitting a form) more than 2 times without success, STOP the automation immediately and RETURN ACTION COMPLETE in ActResult
     d. client_name: the client_name that you are given (as a string)
   - Format the instruction like: "Login using username: X and password: Y. Then [instructions from prompt file]
"""
    )

    print(f"Available tools: {[tool.tool_name for tool in tools]}")

    # Test Invoke Agent
    response = agent("client_name=BlueHonuFire")


Available tools: ['CredentialsRetriever___get_credentials', 'PromptFileRetriever___get_prompt_file', 'nova_act_download']
I'll help you with the BlueHonuFire client request. Let me start by retrieving the necessary information.

First, let's get the login credentials for BlueHonuFire:
Tool #1: CredentialsRetriever___get_credentials
Now, let's get the task instructions from the prompt file:
Tool #2: PromptFileRetriever___get_prompt_file
Thank you! I've retrieved all the necessary information:

**Website Information:**
- URL: https://bluehonufire.com/wp-login.php

**Login Credentials:**
- Username: techresidential
- Password: [REDACTED for security]

**Task Instructions:**
- On the top right corner, find and click the media button
- This will lead to a page with many photos
- Download the first photo that has a sea turtle

Now I'll proceed with the web automation using this information:
Tool #3: nova_act_download



start session 73ab1a42-4344-437a-9367-ae4eb057c7d9 on https://bluehonufire.com/wp-login.php logs dir /var/folders/h_/fl30x8w52y55k21qz1_pl8fm0000gn/T/tmpts1fvxui_nova_act_logs/73ab1a42-4344-437a-9367-ae4eb057c7d9

73ab> act("Login using username: techresidential and password: OHtihVM^P(S!ExITw3. Then on the top right corner, there is a media button, click on it and it will leads you to a lot of photos, download the first photo that has sea turtle.")
73ab> ...
73ab> think("I am on the login page for WordPress. My task is to login using the provided credentials and then download the first photo of a sea turtle from the media page. I see a username field that is empty. I need to type 'techresidential' into the username field to populate it.");
73ab> agentType("techresidential", " <box>241,665,280,935</box> ");
73ab> ...
73ab> think("The username field now contains 'techresidential', so my last action was successful. I see a password field that is empty. I need to type 'OHtihVM^P(S!ExITw3

I apologize for the error that occurred during the web automation. The process timed out while waiting for a download event to complete.

**Summary of what happened:**
- The automation started but encountered a timeout issue (30000ms/30 seconds) while waiting for the file download to complete.
- This could be due to several reasons:
  1. The download might be taking longer than the allotted timeout period
  2. There might be an issue with locating or accessing the sea turtle image
  3. The download button might not have triggered a proper download event

**What was likely completed:**
- Login to the BlueHonuFire website
- Navigation to the media section

**What likely failed:**
- Identifying or downloading the sea turtle image

If you'd like to proceed, I'd recommend either:
1. Trying again with more specific instructions about where the download button might be located
2. Considering a longer timeout for the download process if the files are large
3. Breaking down the task into smalle